In [1]:
import sys
import os
import pandas as pd
import warnings
import re
import datetime

!mkdir 'OpenKE'
! git clone https://github.com/ContentSide/RezoJDM-SDS.git

Cloning into 'RezoJDM-SDS'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 35 (delta 5), reused 34 (delta 4), pack-reused 0
Unpacking objects: 100% (35/35), done.


In [2]:
triplets_train = pd.read_csv("/content/RezoJDM-SDS/datasets/Triplets/triplets_train.csv", index_col=False)
triplets_dev = pd.read_csv("/content/RezoJDM-SDS/datasets/Triplets/triplets_dev.csv", index_col=False)
triplets_test = pd.read_csv("/content/RezoJDM-SDS/datasets/Triplets/triplets_test.csv", index_col=False)
relation_names = pd.read_csv("/content/RezoJDM-SDS/datasets/Triplets/relations_ids_names.csv", index_col=False)

In [3]:
nodes_in_train = set(triplets_train["n1"]).union(set(triplets_train["n2"]))
nodes_in_dev = set(triplets_dev["n1"]).union(set(triplets_dev["n2"]))
nodes_in_test = set(triplets_test["n1"]).union(set(triplets_test["n2"]))

nodes = nodes_in_train.intersection(nodes_in_dev)
nodes = nodes.intersection(nodes_in_test)
nodes = list(nodes)

def remove_delta_nodes(triplets, nodes):
    return triplets[triplets["n1"].isin(nodes) & triplets["n2"].isin(nodes)].reset_index()

triplets_train = remove_delta_nodes(triplets_train, nodes)
triplets_dev = remove_delta_nodes(triplets_dev, nodes)
triplets_test = remove_delta_nodes(triplets_test, nodes)

In [4]:
nodes_in_train = set(triplets_train["n1"]).union(set(triplets_train["n2"]))
nodes_in_dev = set(triplets_dev["n1"]).union(set(triplets_dev["n2"]))
nodes_in_test = set(triplets_test["n1"]).union(set(triplets_test["n2"]))

len(nodes_in_train), len(nodes_in_dev), len(nodes_in_test)

(15525, 15523, 15523)

In [5]:
len(triplets_train["t"].unique()), len(triplets_dev["t"].unique()), len(triplets_test["t"].unique())

(54, 54, 54)

In [6]:
triplets_train.shape[0], triplets_dev.shape[0], triplets_test.shape[0]

(702470, 88304, 88179)

In [10]:
def create_OpenKE_txt(triplets_csv, file_name):

    line = triplets_csv.shape[0]
    text = ""

    text += (str(line) + "\n")

    for _, row in triplets_csv.iterrows():
        text += (str(row["n1"]) +" " +str(row["n2"]) + " "+str(row["t"]) +"\n")
    text = text[:-1]

    output_file = open(file_name,'w')
    output_file.write(text)
    output_file.close()

In [12]:
create_OpenKE_txt(triplets_train,'./OpenKE/train2id.txt')
create_OpenKE_txt(triplets_dev,'./OpenKE/valid2id.txt')
create_OpenKE_txt(triplets_test,'./OpenKE/test2id.txt')

In [13]:
nodes_all = pd.read_csv("/content/RezoJDM-SDS/datasets/Filtered_Nodes_Edges/nodes.csv", index_col=False)

nodes_in_train = set(triplets_train["n1"]).union(set(triplets_train["n2"]))
nodes_in_dev = set(triplets_dev["n1"]).union(set(triplets_dev["n2"]))
nodes_in_test = set(triplets_test["n1"]).union(set(triplets_test["n2"]))

nodes = nodes_in_train.union(nodes_in_dev)
nodes = nodes.union(nodes_in_test)
nodes = list(nodes)

In [14]:
nodes_name = nodes_all[nodes_all["eid"].isin(nodes)].reset_index()[["eid", "n"]]
len(nodes), nodes_all.shape[0]

(15525, 73106)

In [17]:
def create_OpenKE_ent(nodes_name, file_name):
    
    line = nodes_name.shape[0]
    text = ""

    text += (str(line) + "\n")

    for _, row in nodes_name.iterrows():
        text += (row["n"] +"\t" +str(row["eid"])+"\n")
    text = text[:-1]

    output_file = open(file_name,'w')
    output_file.write(text)
    output_file.close()


def create_OpenKE_rel(nodes_name, file_name):
    
    line = nodes_name.shape[0]
    text = ""

    text += (str(line) + "\n")

    for _, row in nodes_name.iterrows():
        text += (row["relation"] +"\t" +str(row["relation_id"])+"\n")
    text = text[:-1]

    output_file = open(file_name,'w')
    output_file.write(text)
    output_file.close()

In [18]:
create_OpenKE_ent(nodes_name, "./OpenKE/entitiy2id.txt")
create_OpenKE_rel(relation_names, "./OpenKE/relation2id.txt")

In [19]:
!zip -r 'OpenKE.zip' 'OpenKE'

  adding: OpenKE/ (stored 0%)
  adding: OpenKE/train2id.txt (deflated 69%)
  adding: OpenKE/relation2id.txt (deflated 53%)
  adding: OpenKE/test2id.txt (deflated 58%)
  adding: OpenKE/valid2id.txt (deflated 58%)
  adding: OpenKE/entitiy2id.txt (deflated 52%)
